<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/02-Word2Vec%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM+Word2Vec

En esta notebook, se entrena y prueba la clasificación de oraciones usando LSTM y Word2Vec pre-entrenado.

El principal beneficio de la incrustación de palabras es que incluso las palabras que no se ven durante el entrenamiento se pueden predecir bien ya que la incrustación de palabras está pre-entrenada con un conjunto de datos más grande que los del dataset actual.


## Carga de librerías, modelo word2vec pre-entrenado y funciones útiles

### Carga de librerías a utilizar para el entrenamiento del modelo

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import sys

# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install gensim
import gensim

### Carga del modelo Word2Vec pre-entrenado de Universidad de Chile

Referencias: https://github.com/dccuchile/spanish-word-embeddings

In [21]:
import os.path
from os import path

if not(path.exists("SBW-vectors-300-min5.bin.gz")):
  !wget http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

# Defino el largo del vector de embedding, en el caso que vamos a usar es 300
VECTOR_EMBEDDINGS = 300

from gensim.models import Word2Vec

filename="SBW-vectors-300-min5.bin.gz"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)
embeddings.init_sims(replace=True)

### Se cargan funciones que se utilizan para el pre-procesamiento de las secuencias

In [22]:
def get_max_length(text):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in text:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def embed(texts): 
  # Inicializo el tensor principal con un array de (1, 250) de ceros
  tensor_principal = tf.convert_to_tensor(np.zeros((1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
  iteration = 0
  for word in texts:  
    try:
      e = tf.convert_to_tensor(np.reshape(embeddings.get_vector(word), (1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
    except:
      e = tf.convert_to_tensor(np.zeros((1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
    if iteration==0:
      tensor_principal = e
    else:
      tensor_principal = tf.concat([tensor_principal, e], 0)
    iteration = iteration+1
    
  return tensor_principal


def get_word2vec_enc(texts):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_texts = []
    for text in texts:
        tokens = text.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_texts.append(word2vec_embedding)
    return encoded_texts
        
def get_padded_encoded_text(encoded_text, max_length):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_text_encoding = []
    for enc_text in encoded_text:

        zero_padding_cnt = max_length - enc_text.shape[0]
        pad = np.zeros((1, VECTOR_EMBEDDINGS))
        for i in range(zero_padding_cnt):
            enc_text = np.concatenate((pad, enc_text), axis=0)
        padded_text_encoding.append(enc_text)
    return padded_text_encoding

def category_encode(category):
    """
    Se encodea la clase en variables dummies
    """
    return pd.get_dummies(category)


def preprocess(x, y, max_length):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    text = x.tolist()
    
    encoded_text = get_word2vec_enc(text)
    padded_encoded_text = get_padded_encoded_text(encoded_text, max_length)
    
    # encoded class
    categorys = y.tolist()
    encoded_category = category_encode(categorys)
    X = np.array(padded_encoded_text)
    Y = np.array(encoded_category)
    return X, Y 

## Carga del dataset y balanceo de clases

In [23]:
# Cargamos el archivo con las consultas que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('03-Correos_variables_estaticas.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv

# Leemos el archivo en un dataframe
import pandas as pd

# Cargamos los datos
df = pd.read_csv('03-Correos_variables_estaticas.csv', delimiter="|")

# Seleccionamos solo la consulta y la clase
x_df = df["Consulta"]
y_df = df["Clase"]

In [24]:
# Definición de la cantidad de clases (el resto se agrupa en OTRAS CONSULTAS)
CANTIDAD_CLASES = 4

# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
clases = y_df.value_counts()
clases_minoritarias = clases.iloc[CANTIDAD_CLASES-1:].keys().to_list()
y_df.loc[y_df.isin(clases_minoritarias)] = "Otras Consultas"

# Se numeriza la clase
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y=le.fit_transform(y_df)

# Me guardo las etiquetas de las clases (numerizadas)
class_list=le.classes_

class_list

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


array(['Boleto Universitario', 'Ingreso a la Universidad',
       'Otras Consultas', 'Requisitos de Ingreso'], dtype=object)

## Separación en train/test

In [25]:
x = x_df
y = y

# Separo datos de entrenamiento y testing
from sklearn.model_selection import train_test_split

# Separo en 80-20 entrenamiento/validación y testeo
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.2)

## Preprocesamiento (codificación del texto a vectores numéricos)

In [26]:
# max_length is used for max sequence of input
max_length = get_max_length(x_train)

train_X, train_Y = preprocess(x_train, y_train, max_length)

## Construcción del Modelo

In [27]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(CANTIDAD_CLASES, activation='softmax'))

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Entrenamiento del modelo

In [29]:
print('Train...')
model.fit(train_X, train_Y,epochs=100)

Train...
Epoch 1/100
25/25 [==============================] - 3s 37ms/step - loss: 1.3619 - accuracy: 0.3593
Epoch 2/100
25/25 [==============================] - 1s 37ms/step - loss: 1.3301 - accuracy: 0.3660
Epoch 3/100
25/25 [==============================] - 1s 37ms/step - loss: 1.2654 - accuracy: 0.4433
Epoch 4/100
25/25 [==============================] - 1s 37ms/step - loss: 1.1364 - accuracy: 0.4974
Epoch 5/100
25/25 [==============================] - 1s 38ms/step - loss: 1.0186 - accuracy: 0.5780
Epoch 6/100
25/25 [==============================] - 1s 37ms/step - loss: 0.9404 - accuracy: 0.6002
Epoch 7/100
25/25 [==============================] - 1s 38ms/step - loss: 0.9040 - accuracy: 0.6367
Epoch 8/100
25/25 [==============================] - 1s 36ms/step - loss: 0.8959 - accuracy: 0.6674
Epoch 9/100
25/25 [==============================] - 1s 37ms/step - loss: 0.8225 - accuracy: 0.6998
Epoch 10/100
25/25 [==============================] - 1s 37ms/step - loss: 0.8162 - accurac

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (32, 32)                  42624     
_________________________________________________________________
dense_1 (Dense)              (32, 4)                   132       
Total params: 42,756
Trainable params: 42,756
Non-trainable params: 0
_________________________________________________________________


### Testeo del Modelo

In [31]:
# max_length is used for max sequence of input
max_length = get_max_length(x_test)

test_X, test_Y = preprocess(x_test, y_test, max_length)


score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

7/7 - 1s - loss: 0.7510 - accuracy: 0.7200
Test score: 0.7510371208190918
Test accuracy: 0.7200000286102295
